In [1]:
#installing the kaggle library
! pip install kaggle

In [2]:
#uploading the kaggle json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
#Import the twitter sentiment analysis using API. API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import os
from zipfile import ZipFile

dataset = '/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip_ref:
    zip_ref.printdir()
with ZipFile(dataset, 'r') as zip_ref:
    zip_ref.extractall('/content/sentiment140_extracted')

# Verify the extraction
extracted_files = os.listdir('/content/sentiment140_extracted')
print(extracted_files)


File Name                                             Modified             Size
training.1600000.processed.noemoticon.csv      2019-09-21 02:36:32    238803811
['training.1600000.processed.noemoticon.csv']


In [5]:
#importing the required dependencies and functions
import numpy as np
import pandas as pd
#re is regular expression for pattern matching and searching through the data
import re
#natural language toolkit library corpus module
from nltk.corpus import stopwords
#PorterStemmer reduces the word to the root word
from nltk.stem.porter import PorterStemmer
#sklearn feature extraction widely used ml library convert the text to numerical data
from sklearn.feature_extraction.text import TfidfVectorizer
#training testing to split the data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
#downloading the stopwords, they are something that do not add any meaning to your textual data like i me myself etc
#from natural language toolkit library
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
column_names=['target','id','date','flag','user','text']
data=pd.read_csv('/content/sentiment140_extracted/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [8]:
data.shape

(1600000, 6)

In [9]:
data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [10]:
data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [11]:
data.replace({'target':{4:'positive'}},inplace=True)

In [12]:
data.replace({'target':{0:'negative'}},inplace=True)

In [13]:
data['target'].value_counts()

target
negative    800000
positive    800000
Name: count, dtype: int64

In [14]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Download stopwords
import nltk
nltk.download('stopwords')

# Initialize the PorterStemmer
port_stemmer = PorterStemmer()

def stemming(content):
    # Remove all characters that are not letters
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    # Split the text into words
    stemmed_content = stemmed_content.split()
    # Stem words and remove stopwords
    stemmed_content = [port_stemmer.stem(word) for word in stemmed_content if word.lower() not in stopwords.words('english')]
    # Join the words back into a single string
    return ' '.join(stemmed_content)

# Example usage
sample_text = "This is an example text to demonstrate the stemming process! Check it out at http://example.com."
print(stemming(sample_text))

# Assuming df is your DataFrame loaded with the dataset
# Assign column names if not already assigned




exampl text demonstr stem process check http exampl com


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
sampled_df = data.sample(frac=0.01, random_state=42)

# Apply the stemming function to the sampled text column
sampled_df['cleaned_text'] = sampled_df['text'].apply(stemming)


In [17]:
  print(sampled_df['cleaned_text'])

541200                             chrishasboob ahhh hope ok
750                        misstoriblack cool tweet app razr
766711     tiannachao know famili drama lame hey next tim...
285055     school email open geographi stuff revis stupid...
705995                                  upper airway problem
                                 ...                        
1326992                                        way last exam
1006015    go check guy pretti freakin kewl http youtub c...
452359     enjoy wonder weekend princess amp famili ask b...
449998     got sore throat well realli sore feel dri time...
987728     upload track new album island life listen pleasur
Name: cleaned_text, Length: 16000, dtype: object


In [18]:
print(sampled_df['target'])

541200     negative
750        negative
766711     negative
285055     negative
705995     negative
             ...   
1326992    positive
1006015    positive
452359     negative
449998     negative
987728     positive
Name: target, Length: 16000, dtype: object


In [19]:
# Feature extraction with sparse matrix
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(sampled_df['cleaned_text'])
y = sampled_df['target']

In [20]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict the sentiment for the test set
y_pred = model.predict(X_test)


In [21]:
# Combine tweets with predictions
result_df = pd.DataFrame({'tweet': sampled_df['text'][y_test.index], 'predicted_sentiment': y_pred})

# Print each tweet with its predicted sentiment
for index, row in result_df.iterrows():
    print(f"Tweet: {row['tweet']}")
    print(f"Sentiment: {row['predicted_sentiment']}")
    print()

Streaming output truncated to the last 5000 lines.
Sentiment: negative

Tweet: @pimme Nu har jag gjort reset, fast datorn hittar liksom inte routern nu...   What's up with that?
Sentiment: positive

Tweet: Making brownies and watching old Grey's episodes. I'm going to miss George. 
Sentiment: negative

Tweet: @brookeburke OMG you gotta a girlfriend?  That is so hot! 
Sentiment: negative

Tweet: Shoutout to @Caramel_tinny she like my older sista gotta love her thx for the help earlier 
Sentiment: positive

Tweet: I wanna have long, wavy hair kind of like Serena's. 
Sentiment: negative

Tweet: My iPhone battery died this AM. 
Sentiment: negative

Tweet: Had to bail on a 16 miler after 6. Thinks he is getting tendinitis in his calves. 
Sentiment: negative

Tweet: buh-bye! bro is gonna use the laptop  tah-tah for now ;)
Sentiment: positive

Tweet: @TennisTim Did you watch the game?  (Cirstea - Jankovic)
Sentiment: positive

Tweet: Everyone follow @JKitsJena because this twitter is fawked, 